In [1]:
import os 
import pickle 
from natsort import natsorted


def load_data_as_dict(directory_path): 
    entries = os.listdir(directory_path)
    files = natsorted([entry for entry in entries if os.path.isfile(os.path.join(directory_path, entry))])
    list_of_dict = []
    list_of_uuid = []
    for file_name in files: 
        file_path = os.path.join(directory_path,file_name)
        with open(file_path, 'rb') as file: 
            dict = pickle.load(file)
            list_of_dict.append(dict)

            uuid = dict['task_dag'].id 
            if uuid not in list_of_uuid:
                list_of_uuid.append(uuid)
    return list_of_dict, list_of_uuid

In [23]:

data_7_unique_dag, _ = load_data_as_dict('data/task_7')
print(f"----KEYS in the DICT----")
for key in (list(data_7_unique_dag[0].keys())): 
    print(key)

print(f"\n----Vars in task_dag----")
for mem_var in list(vars(data_7_unique_dag[0]["task_dag"]).items()):
    print(mem_var)

----KEYS in the DICT----
task_dag
network
map
sim_successfull_flag
network_processing_time
sim_exec_time
lost_packets
avg_flit_lat
avg_packet_lat
avg_network_lat

----Vars in task_dag----
('num_nodes', 7)
('max_out', 1)
('alpha', 0.5)
('beta', 0.0)
('position', {'Start': (0, 5.5), 'Exit': (18, 5.5), 1: (3, 1), 2: (3, 6), 3: (6, 1), 4: (9, 1), 5: (12, 1), 6: (15, 1), 7: (15, 6)})
('id', 'e83b483b-c0ea-4c48-83f9-693ed29d8442')
('graph', <networkx.classes.digraph.DiGraph object at 0x7f2a79d4f280>)
('edge_attr', {(1, 3): 1, (3, 4): 1, (2, 3): 1, (4, 5): 1, (5, 6): 1, (6, 'Exit'): 1, ('Start', 1): 1, ('Start', 2): 1, ('Start', 7): 1, (7, 'Exit'): 1})
('node_attr', {1: 1, 3: 1, 2: 1, 4: 1, 5: 1, 6: 1, 'Start': 1, 7: 1, 'Exit': 1})


In [24]:
# Checking if all Demand and Delay is 
for data in data_7_unique_dag:
    task_graph = data['task_dag']
    print( f"\n"
            f"Demand: {task_graph.edge_attr}"
            f"Delay: {task_graph.edge_attr}"
    )

# Check if two graphs are similar 
    # idea 1 -> comapre edge index (not ideal)
    # idea 2 -> nx.is_isomorphic 


Demand: {(1, 3): 1, (3, 4): 1, (2, 3): 1, (4, 5): 1, (5, 6): 1, (6, 'Exit'): 1, ('Start', 1): 1, ('Start', 2): 1, ('Start', 7): 1, (7, 'Exit'): 1}Delay: {(1, 3): 1, (3, 4): 1, (2, 3): 1, (4, 5): 1, (5, 6): 1, (6, 'Exit'): 1, ('Start', 1): 1, ('Start', 2): 1, ('Start', 7): 1, (7, 'Exit'): 1}

Demand: {(1, 2): 1, (2, 4): 1, (4, 5): 1, (3, 4): 1, (5, 7): 1, (7, 'Exit'): 1, ('Start', 1): 1, ('Start', 3): 1, ('Start', 6): 1, (6, 'Exit'): 1}Delay: {(1, 2): 1, (2, 4): 1, (4, 5): 1, (3, 4): 1, (5, 7): 1, (7, 'Exit'): 1, ('Start', 1): 1, ('Start', 3): 1, ('Start', 6): 1, (6, 'Exit'): 1}

Demand: {(1, 2): 1, (2, 3): 1, (3, 4): 1, (4, 6): 1, (6, 'Exit'): 1, ('Start', 1): 1, ('Start', 5): 1, ('Start', 7): 1, (5, 'Exit'): 1, (7, 'Exit'): 1}Delay: {(1, 2): 1, (2, 3): 1, (3, 4): 1, (4, 6): 1, (6, 'Exit'): 1, ('Start', 1): 1, ('Start', 5): 1, ('Start', 7): 1, (5, 'Exit'): 1, (7, 'Exit'): 1}

Demand: {(1, 3): 1, (3, 'Exit'): 1, (2, 3): 1, ('Start', 1): 1, ('Start', 2): 1, ('Start', 4): 1, ('Start', 5)

['task_dag', 'network', 'map', 'sim_successfull_flag', 'network_processing_time', 'sim_exec_time', 'lost_packets', 'avg_flit_lat', 'avg_packet_lat', 'avg_network_lat']
